<a href="https://colab.research.google.com/github/Biswarup03/Rock-vs-mine-prediction-using-sonar-data/blob/main/rock_vs_mine_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install streamlit pyngrok pandas numpy scikit-learn seaborn matplotlib plotly


In [13]:
!pip install streamlit pyngrok --quiet


In [14]:
from pyngrok import ngrok

# Set your ngrok auth token
!ngrok config add-authtoken 2wa46hdOX73bk4bQWgsTfcn3JBS_7ugj957csFbG6STeGHDvH


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [15]:
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    auc
)

# -----------------------------
# Load Dataset
# -----------------------------
df = pd.read_csv("Copy of sonar data.csv", header=None)

X = df.drop(columns=60, axis=1)
y = df[60]

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, stratify=y, random_state=1
)

# Train model
model = LogisticRegression()
model.fit(X_train, y_train)

# -----------------------------
# Streamlit UI
# -----------------------------
st.set_page_config(page_title="Rock vs Mine Prediction", page_icon="🔍", layout="wide")

st.title("🔍 Rock vs Mine Prediction Dashboard")
st.markdown("This app uses **Logistic Regression** to classify sonar signals into **Rock (R)** or **Mine (M)**.")

# Sidebar
st.sidebar.header("📊 Model Information")
train_acc = accuracy_score(model.predict(X_train), y_train)
test_acc = accuracy_score(model.predict(X_test), y_test)
st.sidebar.write(f"**Training Accuracy:** {train_acc:.2f}")
st.sidebar.write(f"**Testing Accuracy:** {test_acc:.2f}")

# Tabs
tab1, tab2, tab3, tab4 = st.tabs(["📂 Bulk CSV Prediction", "🧩 Single Prediction", "📈 Model Evaluation", "📊 Dataset Overview"])

# -----------------------------
# TAB 1: BULK CSV PREDICTION
# -----------------------------
with tab1:
    st.subheader("📂 Upload CSV for Bulk Predictions")
    uploaded_file = st.file_uploader("Upload a CSV file with 60 features", type=["csv"])

    if uploaded_file is not None:
        new_data = pd.read_csv(uploaded_file, header=None)

        if new_data.shape[1] != 60:
            st.error("❌ CSV must have exactly 60 columns!")
        else:
            st.success("✅ File uploaded successfully!")

            predictions = model.predict(new_data)
            probabilities = model.predict_proba(new_data)

            results = pd.DataFrame(new_data, columns=[f"Feature {i+1}" for i in range(60)])
            results["Prediction"] = predictions
            results["Prediction"] = results["Prediction"].map({"R": "🪨 Rock", "M": "💣 Mine"})
            results["Rock Probability"] = probabilities[:, 0]
            results["Mine Probability"] = probabilities[:, 1]

            st.dataframe(results)

            csv = results.to_csv(index=False).encode("utf-8")
            st.download_button(
                label="⬇️ Download Predictions",
                data=csv,
                file_name="sonar_predictions.csv",
                mime="text/csv"
            )

# -----------------------------
# TAB 2: SINGLE PREDICTION (WITH 🎲 RANDOM FILL)
# -----------------------------
with tab2:
    st.subheader("🧩 Single Prediction Mode")
    st.write("Enter 60 sonar signal values manually or click **🎲 Random Fill** to auto-fill values:")

    # Session state to store inputs
    if "input_values" not in st.session_state:
        st.session_state.input_values = [0.0] * 60

    # 🎲 Random Fill Button
    if st.button("🎲 Random Fill"):
        st.session_state.input_values = np.round(np.random.rand(60), 2).tolist()

    # Input fields in 6 columns
    cols = st.columns(6)
    input_data = []
    for i in range(60):
        value = cols[i % 6].number_input(
            f"F{i+1}",
            min_value=0.0,
            max_value=1.0,
            step=0.01,
            value=st.session_state.input_values[i]
        )
        input_data.append(value)

    # Prediction Button
    if st.button("🔎 Predict"):
        input_array = np.asarray(input_data).reshape(1, -1)
        prediction = model.predict(input_array)
        probability = model.predict_proba(input_array)[0]

        st.subheader("Prediction Result")
        if prediction[0] == "R":
            st.success(f"🪨 The object is **Rock** (Probability: {probability[0]:.2f})")
        else:
            st.error(f"💣 The object is **Mine** (Probability: {probability[1]:.2f})")

        # Probability Bar Chart
        fig = go.Figure(data=[
            go.Bar(name="Probability", x=["Rock", "Mine"], y=probability, marker_color=["#4CAF50", "#FF5733"])
        ])
        fig.update_layout(title="Prediction Probability", yaxis=dict(title="Probability"))
        st.plotly_chart(fig)

# -----------------------------
# TAB 3: MODEL EVALUATION
# -----------------------------
with tab3:
    st.subheader("📈 Model Evaluation Metrics")

    # Confusion Matrix
    st.write("### Confusion Matrix")
    cm = confusion_matrix(y_test, model.predict(X_test))
    fig, ax = plt.subplots()
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Rock", "Mine"], yticklabels=["Rock", "Mine"], ax=ax)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    st.pyplot(fig)

    # Classification Report
    st.write("### Classification Report")
    report = classification_report(y_test, model.predict(X_test), target_names=["Rock", "Mine"], output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    st.dataframe(report_df)

    # ROC Curve
    st.write("### ROC Curve")
    y_prob = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test.map({"R": 0, "M": 1}), y_prob)
    roc_auc = auc(fpr, tpr)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=fpr, y=tpr, mode="lines", name=f"ROC Curve (AUC={roc_auc:.2f})"))
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode="lines", line=dict(dash="dash"), name="Random Guess"))
    fig.update_layout(title="ROC Curve", xaxis=dict(title="False Positive Rate"), yaxis=dict(title="True Positive Rate"))
    st.plotly_chart(fig)

# -----------------------------
# TAB 4: DATASET OVERVIEW
# -----------------------------
with tab4:
    st.subheader("📊 Dataset Overview")
    st.dataframe(df.head())
    st.write("### Class Distribution")
    st.bar_chart(df[60].value_counts())


Overwriting app.py


In [18]:
!streamlit run app.py --server.port 8501 &>/dev/null &


In [19]:
from pyngrok import ngrok

# Kill existing tunnels first (just in case)
ngrok.kill()

# Create new tunnel
public_url = ngrok.connect(8501)
print("🚀 Your Streamlit app is live here:", public_url)


🚀 Your Streamlit app is live here: NgrokTunnel: "https://499141645c9c.ngrok-free.app" -> "http://localhost:8501"


To authenticate `pyngrok` with ngrok, you need an authtoken. You can get one from your ngrok dashboard: https://dashboard.ngrok.com/get-started/your-authtoken

Once you have your authtoken, add it to Colab's secrets manager under the "🔑" icon in the left panel. Name the secret `NGROK_AUTH_TOKEN`.

Then, run the following cell to set up the authtoken in your environment: